# clean-homelessness script

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import intake
import boto3

catalog = intake.open_catalog('../catalogs/*.yml')
s3 = boto3.client('s3')
bucket_name = 's3://city-of-los-angeles-data-lake/public-health-dashboard/'

/opt/conda/lib/python3.7/site-packages/intake/source/discovery.py:136: FutureWarning: The drivers ['geojson', 'postgis', 'shapefile', 'spatialite'] do not specify entry_points and were only discovered via a package scan. This may break in a future release of intake. The packages should be updated.
  FutureWarning)


## Merge homelessness with clipped census tracts

In [2]:
df = gpd.read_file(f'{bucket_name}gis/raw/homelessness_2017_2019.geojson')

In [3]:
tracts = gpd.read_file(f'{bucket_name}gis/raw/census_tracts.geojson')

In [5]:
pyarrow.__version__

NameError: name 'pyarrow' is not defined

In [4]:
pop = pd.read_parquet(f'{bucket_name}data/raw/pop_by_tract2017.parquet') 

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
pyarrow or fastparquet is required for parquet support

In [ ]:
final = pd.merge(df, tracts, on = 'GEOID', how = 'inner', validate = 'm:1')

# Only keep tracts that are in City of LA
final = final[final.CD != 0]

final = final.drop(columns = ['Tract', 'geometry_x'])
final.rename(columns = {'geometry_y': 'geometry'}, inplace = True)

final = final.reindex(columns=['GEOID', 'SPA', 'SD', 'CD', 'year', 
                               'totUnshelt', 'totShelt', 'totPeople', 'full_area', 'clipped_area', 'geometry'])

In [ ]:
# Export to S3
final.to_file(driver = 'GeoJSON', filename = '../gis/homelessness_la.geojson')
s3.upload_file('../gis/homelessness_la.geojson', 'city-of-los-angeles-data-lake', 
               'public-health-dashboard/gis/raw/homelessness_lacity_2017_2019.geojson')